In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import string
from clean_data import*

In [2]:
candidate=DataFrame(columns=["id", "name","state_fk","date_birth", "place_birth", "position", "URL_photo", "party"])

candidate1=pd.DataFrame([[1, "Don Blankenship",1, "March 14, 1950", "Stopover, Kentucky", "Coal magnate", "https://i2.wp.com/woay.tv/wp-content/uploads/2017/05/GTY_don_blankenship_jt_160406_4x3_992.jpg?resize=735%2C400&ssl=1","Republican" ],
                     [2, "Bo Copley",1, "", " Dingess, West Virginia", "","https://heavyeditorial.files.wordpress.com/2017/05/bo-copley-e1494012597984.jpg?quality=65&strip=all&w=780&strip=all","Republican" ],
                     [3, "Evan Jenkins",1, "September 12, 1960", "Huntington, West Virginia","U.S. House of Representatives","https://www.congress.gov/img/member/114_rp_wv_3_jenkins_evan.jpg","Republican"  ],
                     [4, "Joe Manchin",1, " August 24, 1947", "Farmington, West Virginia", "United States Senator", "https://fi.wikipedia.org/wiki/Joe_Manchin#/media/File:Senator_Manchin.jpg", "Democrat"],
                     [5, "Patrick Morrisey",1,"December 21, 1967","Brooklyn, New York City, NY","West Virginia Attorney General since 2013","http://wvmetronews.com/wordpress/wp-content/uploads/2016/02/Patrick-Morrisey.jpg?x43308","Republican"  ],
                     [6, "Jack Newbrough",1, "", "","U.S. Navy veteran","http://s3.amazonaws.com/ogden_images/www.weirtondailytimes.com/images/2017/09/23190845/Newbrough-003-1100x825.jpg" ,"Republican" ],
                     [7, "Paula Jean Swearengin", 1, "born 1973/74","Mullens, West Virginia","","https://cbbba9d899c5f8ae6108-e379174d4d25dcb78174c6cab0a88cea.ssl.cf1.rackcdn.com/1275564-l.jpg", "Democrat"],
                     [8, "Tom Willis",1, "","","","https://api.ballotpedia.org/v3/thumbnail/200/300/crop/best/2C512069-7298-4BAC-B9F3-6BFECDC2FB57.jpeg","Republican" ],
                     [9, "Nicholas Freitas", 2, "", "", "","","Republican"],
                     [10, "E. W. Jackson", 2,"","","","","Republican"],
                     [11, "Tim Kaine", 2, "", "", "", "", "Democrat"],
                     [12, "Corey Stewart", 2, "", "", "", "", "Republican"],
                     [13, "Matt Waters", 2, "", "", "", "", "Libertarian"],
                     [14, "Carl Bible", 3, "", "", "", "", "Independent"],
                     [15, "Ted Cruz", 3, "", "", "", "", "Republican"],
                     [16, "Bob McNeil", 3, "", "", "", "", "American Citizen Party"],
                     [17, "Beto O'Rourke", 3, "", "", "", "", "Democrat"]],
                        columns=["id", "name","state_fk","date_birth", "place_birth", "position","URL_photo", "party"])
candidate=candidate.append(candidate1)


In [3]:
all_candidates = []
for index,row in candidate.iterrows():
    all_candidates.append(row['name'])

In [4]:
structure_data(all_candidates, "article_data")

In [ ]:
for i in range(1,10):
    print(i)
    if(i > 5):
        break
"April".lower() in ["APRIL", "nov"]

In [ ]:
a = [[1,2,3,4,5,6],[7,8,9]]



In [ ]:
b = [[2*x for y in a] for x in y]
b


In [ ]:
c = [[2*x for x in y] for y in a]
#[[int(x) for x in feet] for x in feet]
c

In [ ]:
"hello".replace('h','')

In [ ]:
okay = {1:"a", 2:"b", 3:"c"}

In [ ]:
for k in okay.keys():
    print(k)

In [ ]:
if int('5') in [1,2,5]:
    print(1)

In [ ]:
a[0][0:-1]

In [ ]:
for i in range(len(a[0][0:-1])):
    print(i)

In [ ]:
next_try = ["a", "b", "v"]
reform = ",".join(next_try)
reform